In [1]:
!pip install tensorflow_decision_forests -U -qq
!pip install wurlitzer -U -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 20.7 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow_decision_forests as tfdf

In [3]:
#@title Define "set_cell_height".

from IPython.core.magic import register_line_magic
from IPython.display import Javascript
from IPython.display import display

# Some of the model training logs can cover the full
# screen if not compressed to a smaller viewport.
# This magic allows setting a max height for a cell.
@register_line_magic
def set_cell_height(size):
  display(
      Javascript("google.colab.output.setIframeHeight(0, true, {maxHeight: " +
                 str(size) + "})"))

In [10]:
%%time
%set_cell_height 300

# Load your data
data = pd.read_csv("/content/Full_AM(4).csv", delimiter='\t', encoding='utf-8')

# Set the value for k (number of folds)
k = 5

# Initialize KFold cross-validator
kf = KFold(n_splits=k, shuffle=True, random_state=42)

model_config = {
    "task": tfdf.keras.Task.REGRESSION,
    "num_trees": 430,
    "max_depth": 16,
    "min_examples": 1,
    "num_candidate_attributes_ratio": 0.5,
}

# Initialize a machine learning model
model = tfdf.keras.RandomForestModel(**model_config)
# Create lists to store performance metrics
mse_scores = []

# Цей цикл проходить через кожен фолд, де fold - номер фолда, а train_index і val_index - індекси навчального
# і валідаційного наборів даних для поточного фолда.
for fold, (train_index, val_index) in enumerate(kf.split(data)):

    # Дані розбиваються на навчальний (fold_train_data) і валідаційний (fold_val_data) набори на основі поточних індексів.
    fold_train_data, fold_val_data = data.iloc[train_index], data.iloc[val_index]

    # Convert training data to TensorFlow dataset
    fold_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(fold_train_data, label="N_Fe", task=model.task)

    # Convert validation data to TensorFlow dataset
    fold_val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(fold_val_data, label="N_Fe", task=model.task)

    # Train the model on the training set
    model.fit(fold_train_ds)

    # Make predictions on the validation set
    predictions = model.predict(fold_val_ds)

    # Evaluate the model performance using Mean Squared Error
    mse = mean_squared_error(fold_val_data["N_Fe"], predictions)
    mse_scores.append(mse)

    print(f"Fold {fold + 1}: Mean Squared Error - {mse}")

overall_mean_mse = np.mean(mse_scores)
overall_std_mse = np.std(mse_scores)
print(f"\nOverall Mean Squared Error: {overall_mean_mse}")
print(f"Standard Deviation of Mean Squared Error: {overall_std_mse}")

for fold, mse in enumerate(mse_scores):
    print(f"\n Fold {fold + 1}: Mean Squared Error - {mse}")

<IPython.core.display.Javascript object>

Use /tmp/tmpo0pow5pe as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.239915. Found 13519 examples.
Training model...
Model trained in 0:00:23.023725
Compiling model...
Model compiled.
4/4 [==============================] - 0s 76ms/step
Fold 1: Mean Squared Error - 0.05513292145697236
Reading training dataset...
Training dataset read in 0:00:00.054700. Found 13519 examples.
Training model...
Model trained in 0:00:24.140443
Compiling model...
Model compiled.
4/4 [==============================] - 1s 84ms/step
Fold 2: Mean Squared Error - 0.05819690245974869
Reading training dataset...
Training dataset read in 0:00:00.054151. Found 13519 examples.
Training model...
Model trained in 0:00:23.598182
Compiling model...
Model compiled.
4/4 [==============================] - 0s 84ms/step
Fold 3: Mean Squared Error - 0.0462986385983367
Reading training dataset...
Training dataset read in 0:00:00.054984. Found 13519 examples.
Training model...
Model tr